<a href="https://colab.research.google.com/github/pschorey/Valpo_IT533/blob/main/04_Pedagogy_Web_Scraping_DNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [ ]:
url = "https://www.letssingit.com/miley-cyrus-lyrics-flowers-d7nsn9k"
#url = "https://www.letssingit.com/songs/popular/1" #top 1000
html = urlopen(url)

In [ ]:
soup = BeautifulSoup(html, 'lxml')
type(soup)

bs4.BeautifulSoup

In [ ]:
title = soup.title
print(title)

<title>Miley Cyrus - Flowers Lyrics | LetsSingIt Lyrics</title>


In [ ]:
text = soup.get_text()
print(soup.text)




Miley Cyrus - Flowers Lyrics | LetsSingIt Lyrics


















































 Home
 Charts
 Forum


 Comments
 Trending topics
 New topics
 Browse
 Moderators


 Help
 About us


 LetsSingIt
 Contact
 Museum
 Sitemap
 usage







the internet lyrics database


en 



Choose your language

LetsSingIt comes to you in your own language! Choose your language below. Not all languages are fully translated. Help us translate the rest!
Help translate






5

 



Hi guest, welcome to LetsSingIt!



                  You're not logged in.
                  Log in to enjoy extra privileges that come with a free membership!
                  read more





log in
register





































"Flowers" Lyrics
Miley Cyrus



 4.1 / 5






overview
albums (21)
lyrics (375)
submit






›Miley Cyrus›lyrics›Flowers




 Song updated, review now!

We were good, we were gold
Kinda dream that can't be sold
We were right 'til we weren't
Built a home and watched

In [ ]:

#song <a href="..." class="high_profile">Från Och Med Du lyrics</a> #all links have class="high_profile"
#navigation <a class="pagination_current" href="1">1</a> #notice href="1" -ordinal numbers, can be looped through

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/My Drive/output.csv'
with open(path, 'w', encoding = 'utf-8-sig') as f:
  df.to_csv(f)